### Transmission:
We encode the 80 bytes textfile bit(byte) by bit(byte). <br>
Each bit(byte) is encoded in a 1(128)-bits codeword=column of matrix M, with a + or - sign. <br>
We form an intermediary 80(640) bits vector X_i (for byte i) that consists of 80(5) repetitions of the codeword. <br>
Once we've done this for all bytes in the textfile, we concatenate all intermediary vectors X_i to obtain a 640 * 80=51200 bits vector X.

##### Byte to codeword mapping:
   Let k be the bit to encode. If k=1 then codeword = +1, if k=0 then codeword = -1.

   (Let k be the value represented by the byte. <br>
   Let l = k mod 128. <br>
   The corresponding codeword is the l-th column of matrix M, with a + sign if l<= 128, - sign if l>128.)

### Reception:
We receive a 51200 bits vector Y = X + Z, with Z_i iid N(0,11). <br>
Divide Y in 51200/80 intermediary subvectors Y_i of length 640 bits. <br>
...


In [1]:
import numpy as np
import math

In [2]:
# GLOBAL VARIABLES
# text of 80 chars -> 640 bits
# we encode each bit into 51200/640 = 80 bits
# each byte is encoded with 80*8 bits = 640 bits -> 2^9 

N = 51200
M_COLUMN_SIZE = 128 #if m = 2⁸, m = 2n => n = 2⁷
REPETITION = 5
INTERMEDIARY_SIZE = M_COLUMN_SIZE*REPETITION #128*5

# with CODEWORD_SIZE = 2^i
i = 7

# M is CODEWORD_SIZE * CODEWORD_SIZE, we want sqrt(EPSILON/CODEWORD_SIZE) = 1
EPSILON = M_COLUMN_SIZE #s(never used)


# M is a matrix 2^8 rows *2^8 columns 
# we encode 8 bits so 2^8=256 values possible 
# there are 512 possible codewords so we can make a repetition code 
# bytes go from 0 to 256 
# we can map to one of the 2^8 columns with a + or - sign 
# if the number is even we encode B(= the binary value of the byte) to column B with +sign of M  
# if the number is odd we encode B(= the binary value of the byte) to column B with -sign of M  

### but this means we never use the codeword column +B, where B is odd, idem for -B when B even.

## Transmission

In [3]:
def compute_M(j):
    if j== 0:
        return np.array([1])
    M_1 = np.array([[1,1],[1,-1]])
    temp = M_1
    for l in range (j-1):
        up = np.concatenate((temp,temp), axis=1) 
        bottom = np.concatenate((temp,-temp), axis=1)
        temp = np.concatenate((up,bottom), axis=0)
    return temp

In [4]:
def columnk_M(M,k):
    if M.shape == (1,):
        return M[0]
    if k>=0 and k<np.size(M, 1):
        return M[:,k-1]
    else:
        print("bad index for column: "+str(k))


In [5]:
def encode_mk(M,k,sign):
    return (sign*columnk_M(M,k))

In [7]:
def transmitter3(filename):
    
    f=open(filename, 'rb')
    data = f.read()
    f.close()
    X = np.array([])
    sign = 1
    for j in range (len(data)):
        
        # get the byte
        if (data[j]<=M_COLUMN_SIZE):
            sign = 1
        else :
            sign = -1
        
        l = np.mod(data[j], M_COLUMN_SIZE)
            
        # create intermediary REPETITION*M_COLUMN_SIZE bits X_i
        X_i = np.array([])
        for r in range (REPETITION):
            V = encode_mk(compute_M(i),l,sign)
            X_i = np.concatenate((X_i,V), axis=0)
            
        # append intermediary X_i to N-bits vector X
        X = np.concatenate((X,X_i), axis=0)

    return np.savetxt("signalX.txt", X, fmt="%s")
    
    

In [8]:
transmitter3("pdc_input_alphabet.txt")

    python3 client.py --input_file=signalX.txt --output_file=output.txt --srv_hostname=iscsrv72.epfl.ch --srv_port=80 

## Reception

In [38]:
# Y_i is a INTERMEDIARY_SIZE-bits subvector of N-bits Y.
# j should correspond to the global var i

def compute_U(M,Y_i,j): 
    if j==0:
        return Y_i
    if j==1:
        return np.dot(M,Y_i)
    U_bis = compute_M(j-1)
    U_t = np.dot(U_bis,Y_i[0:int(M_COLUMN_SIZE/2)])
    U_b = np.dot(U_bis,Y_i[int(M_COLUMN_SIZE/2):])
    return np.concatenate((U_t+U_b,U_t-U_b),axis=0)   

In [39]:
def decode_U(U): # returns the value of the decoded text byte
    U_bis=abs(U)
    k=np.argmax(U_bis)
    if U[int(k)]<0:
        return -(k+1)
    else :
        return k+1

In [33]:
# import the names of the months from month.txt as a numpy array
#months = np.genfromtxt("pdc_input_alphabet.txt", dtype='byte') 
#array = np.array([])
f=open("pdc_input_alphabet.txt", 'rb')
Y = f.read(1)
#type(Y)
arr = np.array([])
while Y:
    arr = np.append(arr, Y[0])
    Y = f.read(1)
arr

array([ 97., 122., 101., 114., 116., 121., 117., 105., 111., 112., 113.,
       115., 100., 102., 103., 104., 106., 107., 108., 109., 119., 120.,
        99., 118.,  98., 110.,  97., 122., 101., 114., 116., 121., 117.,
       105., 111., 112., 113., 115., 100., 102., 103., 104., 106., 107.,
       108., 109., 119., 120.,  99., 118.,  98., 110.,  97., 122., 101.,
       114., 116., 121., 117., 105., 111., 112., 113., 115., 100., 102.,
       103., 104., 106., 107., 108., 109., 119., 120.,  99., 118.,  98.,
       110.,  97., 122.,  10.])

In [40]:
def receiver(filename):
    
    # open filename to get corresponding binary vector (N-bits) Y
    f=open(filename, 'rb')
    Y = f.read(1)    
    Y_arr = np.array([])
    while Y:
        Y_arr = np.append(arr, Y[0])
        Y = f.read(1)
    f.close()
    
    output = open("decoded_text.txt", 'wb')
    
    Y_0 = np.array([])
    Y_1 = np.array([])
    Y_2 = np.array([])
    Y_3 = np.array([])
    
    # for all subvector (INTERMEDIARY_SIZE) Y_i in Y, we decode the corresponding byte
    for j in range(0,80, INTERMEDIARY_SIZE):
        Y_i = Y_arr[j:j+INTERMEDIARY_SIZE]
        #print (Y_i)
        Y_0 = Y_i[0:128]
        Y_1 = Y_i[128:256]
        Y_2 = Y_i[256:384]
        Y_3 = Y_i[384:512]
        #print (Y_0)
        #print (Y_1)
        #print (Y_2)
        #print (Y_3)
        print (compute_U(compute_M(i), Y_0, i))
        curr_byte0 = decode_U(compute_U(compute_M(i), Y_0, i))
        curr_byte1 = decode_U(compute_U(compute_M(i), Y_1, i))
        curr_byte2 = decode_U(compute_U(compute_M(i), Y_2, i))
        curr_byte3 = decode_U(compute_U(compute_M(i), Y_3, i))
        print (curr_byte0)
        print (curr_byte1)
        print (curr_byte2)
        print (curr_byte3)

        # each time we get a byte, we write it in a new output file
        output.write(curr_byte)
        
    output.close()
        

In [41]:
receiver("output.txt")

ValueError: shapes (64,64) and (0,) not aligned: 64 (dim 1) != 0 (dim 0)

## All together:

In [43]:
M=compute_M(2)
print(M)


[[ 1  1  1  1]
 [ 1 -1  1 -1]
 [ 1  1 -1 -1]
 [ 1 -1 -1  1]]


In [45]:
columnk_M(M,2)

array([ 1, -1,  1, -1])

In [47]:
M2=compute_M(0)
print(M2.shape)

(1,)


In [48]:
print(encode_mk(M,3,-1))

[-1 -1  1  1]


In [50]:
M1=compute_M(i)
U1=compute_U(M1,encode_mk(M1,1,-1),i)

In [52]:
decode_U(U1)

-1

In [57]:
f = open("pdc_input_alphabet.txt", 'rb')
Y = f.read(2)
f.close()
Y

b'az'